Imports

In [1]:
!pip install scikit-learn
!pip install --upgrade imbalanced-learn --quiet
!pip install imblearn
!pip install numpy
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter

from imblearn.over_sampling import SMOTE

df = pd.read_csv('/kaggle/input/diabetes/diabetic_data.csv')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 78.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


2025-05-13 12:16:16.535391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747138576.802996      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747138576.879642      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Viewing the dataset

In [2]:
# Display general information about the dataset
print("Information of Dataset:")
display(df.info())
print("\n")

# Display descriptive statistics for numeric features
print("Description of Dataset for Numeric Features:")
display(df.describe().T.style.background_gradient(cmap='Blues'))
print("\n")

# Display descriptive statistics for categorical features
print("Description of Dataset for Categorical Features:")
display(df.describe(exclude=np.number).T.style.background_gradient(cmap='Blues'))
print("\n")

# Display percentage of null values in the dataset
print("Null Values in Dataset:")
display(df.isnull().mean() * 100)
print("\n")

# Display duplicate rows in the dataset
print("Duplicate Rows in Dataset:")
display(df[df.duplicated()].style.background_gradient(cmap='Blues'))

Information of Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 no

None



Description of Dataset for Numeric Features:


,count,mean,std,min,25%,50%,75%,max
encounter_id,101766.000000,165201645.622978,102640295.983458,12522.000000,84961194.000000,152388987.000000,230270887.500000,443867222.000000
patient_nbr,101766.000000,54330400.694947,38696359.346534,135.000000,23413221.000000,45505143.000000,87545949.750000,189502619.000000
admission_type_id,101766.000000,2.024006,1.445403,1.000000,1.000000,1.000000,3.000000,8.000000
discharge_disposition_id,101766.000000,3.715642,5.280166,1.000000,1.000000,1.000000,4.000000,28.000000
admission_source_id,101766.000000,5.754437,4.064081,1.000000,1.000000,7.000000,7.000000,25.000000
time_in_hospital,101766.000000,4.395987,2.985108,1.000000,2.000000,4.000000,6.000000,14.000000
num_lab_procedures,101766.000000,43.095641,19.674362,1.000000,31.000000,44.000000,57.000000,132.000000
num_procedures,101766.000000,1.339730,1.705807,0.000000,0.000000,1.000000,2.000000,6.000000
num_medications,101766.000000,16.021844,8.127566,1.000000,10.000000,15.000000,20.000000,81.000000
number_outpatient,101766.000000,0.369357,1.267265,0.000000,0.000000,0.000000,0.000000,42.000000




Description of Dataset for Categorical Features:


,count,unique,top,freq
race,101766,6,Caucasian,76099
gender,101766,3,Female,54708
age,101766,10,[70-80),26068
weight,101766,10,?,98569
payer_code,101766,18,?,40256
medical_specialty,101766,73,?,49949
diag_1,101766,717,428,6862
diag_2,101766,749,276,6752
diag_3,101766,790,250,11555
max_glu_serum,5346,3,Norm,2597




Null Values in Dataset:


encounter_id                 0.000000
patient_nbr                  0.000000
race                         0.000000
gender                       0.000000
age                          0.000000
weight                       0.000000
admission_type_id            0.000000
discharge_disposition_id     0.000000
admission_source_id          0.000000
time_in_hospital             0.000000
payer_code                   0.000000
medical_specialty            0.000000
num_lab_procedures           0.000000
num_procedures               0.000000
num_medications              0.000000
number_outpatient            0.000000
number_emergency             0.000000
number_inpatient             0.000000
diag_1                       0.000000
diag_2                       0.000000
diag_3                       0.000000
number_diagnoses             0.000000
max_glu_serum               94.746772
A1Cresult                   83.277322
metformin                    0.000000
repaglinide                  0.000000
nateglinide 



Duplicate Rows in Dataset:


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted


Handle Missing Values

In [3]:
#max_glu_serum ,weight and A1Cresult has more than 85% null values so reoving it.
# examide and citoglipton has only one value so also removing it.

df.drop(columns=['max_glu_serum','A1Cresult','weight','citoglipton','examide','payer_code','medical_specialty','encounter_id','patient_nbr'],inplace=True)

# Replace null values with the most frequent value in each column
for col in df.columns:
    if df[col].isnull().any():
        most_frequent_value = df[col].mode()[0]
        df[col].fillna(most_frequent_value, inplace=True)

print(df['race'].value_counts())

#replacing ? occurencies
df['race'] = df['race'].apply(lambda x: 'Other' if x == '?' else x)

# removing null values from diag_1,2 and 3.
index=[]
index=list(df[df['diag_1']=='?'].index)
index.extend(df[df['diag_2']=='?'].index)
index.extend(df[df['diag_3']=='?'].index)
df.drop(index=index,inplace=True)


race
Caucasian          76099
AfricanAmerican    19210
?                   2273
Hispanic            2037
Other               1506
Asian                641
Name: count, dtype: int64


In [4]:
#Encoding
le = LabelEncoder()
df['admission_type_id'] = le.fit_transform(df['admission_type_id'])
df['discharge_disposition_id'] = le.fit_transform(df['discharge_disposition_id'])
df['admission_source_id'] = le.fit_transform(df['admission_source_id'])
df['race'] = le.fit_transform(df['race'])
df['gender'] = le.fit_transform(df['gender'])
df['age'] = le.fit_transform(df['age'])
df['diag_1'] = le.fit_transform(df['diag_1'])
df['diag_2'] = le.fit_transform(df['diag_2'])
df['diag_3'] = le.fit_transform(df['diag_3'])
df['metformin'] = le.fit_transform(df['metformin'])
df['repaglinide'] = le.fit_transform(df['repaglinide'])
df['nateglinide'] = le.fit_transform(df['nateglinide'])
df['chlorpropamide'] = le.fit_transform(df['chlorpropamide'])
df['glimepiride'] = le.fit_transform(df['glimepiride'])
df['acetohexamide'] = le.fit_transform(df['acetohexamide'])
df['glipizide'] = le.fit_transform(df['glipizide'])
df['glyburide'] = le.fit_transform(df['glyburide'])
df['tolbutamide'] = le.fit_transform(df['tolbutamide'])
df['pioglitazone'] = le.fit_transform(df['pioglitazone'])
df['rosiglitazone'] = le.fit_transform(df['rosiglitazone'])
df['acarbose'] = le.fit_transform(df['acarbose'])
df['miglitol'] = le.fit_transform(df['miglitol'])
df['troglitazone'] = le.fit_transform(df['troglitazone'])
df['tolazamide'] = le.fit_transform(df['tolazamide'])
df['insulin'] = le.fit_transform(df['insulin'])
df['glyburide-metformin'] = le.fit_transform(df['glyburide-metformin'])
df['glipizide-metformin'] = le.fit_transform(df['glipizide-metformin'])
df['glimepiride-pioglitazone'] = le.fit_transform(df['glimepiride-pioglitazone'])
df['metformin-rosiglitazone'] = le.fit_transform(df['metformin-rosiglitazone'])
df['metformin-pioglitazone'] = le.fit_transform(df['metformin-pioglitazone'])
df['change'] = le.fit_transform(df['change'])
df['diabetesMed'] = le.fit_transform(df['diabetesMed'])


#Transforming in a binary problem
df['readmitted']=df['readmitted'].apply(lambda x : 1 if x=='<30' else 0)
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,2,0,1,0,0,6,3,59,0,18,...,0,3,1,0,0,0,0,0,1,0
2,0,0,2,0,0,6,2,11,5,13,...,0,1,1,0,0,0,0,1,1,0
3,2,1,3,0,0,6,2,44,1,16,...,0,3,1,0,0,0,0,0,1,0
4,2,1,4,0,0,6,1,51,0,8,...,0,2,1,0,0,0,0,0,1,0
5,2,1,5,1,0,1,3,31,6,16,...,0,2,1,0,0,0,0,1,1,0


Building the model


In [5]:

#Dataset unbalanced, using smote
sm=SMOTE(sampling_strategy='minority',random_state=42)

X = df.drop('readmitted', axis=1).values
y = df['readmitted'].values
X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train_cat = to_categorical(y_train, num_classes=2)
y_test_cat = to_categorical(y_test, num_classes=2)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train_cat,
          epochs=20,
          batch_size=64,
          validation_split=0.1)

loss, accuracy = model.evaluate(X_test, y_test_cat)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


2025-05-13 12:16:40.900460: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7816 - loss: 0.4542 - val_accuracy: 0.8189 - val_loss: 0.3910
Epoch 2/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8327 - loss: 0.3668 - val_accuracy: 0.8266 - val_loss: 0.3757
Epoch 3/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8426 - loss: 0.3517 - val_accuracy: 0.8319 - val_loss: 0.3721
Epoch 4/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8467 - loss: 0.3448 - val_accuracy: 0.8407 - val_loss: 0.3553
Epoch 5/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8514 - loss: 0.3350 - val_accuracy: 0.8402 - val_loss: 0.3577
Epoch 6/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8555 - loss: 0.3286 - val_accuracy: 0.8430 - val_loss: 0.3533
Epoch 7/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8621 - loss: 0.3181 - val_accuracy: 0.8459 - val_loss: 0.3475
Epoch 8/20
2003/2003 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8612 - loss: 0.3179 - 